In [80]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.impute import SimpleImputer

In [3]:
laptop_data=pd.read_csv("laptop_price.csv",encoding="latin-1")

In [27]:
laptop_data.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


In [28]:
laptop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


In [72]:
laptop_data.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [95]:
#testing
def transformer(dataset):
    wt=[]
    ram=[]
    x_res=[]
    y_res=[]
    all_res=[]
    scrn_type=[]
    cpu_freq=[]
    cpu_type=[]
    for i in [
       'ScreenResolution', 'Cpu', 'Ram', 'Gpu',  'Weight',
       ]:
        X=dataset[i]
        if 'kg' in X[0]:
            for i in X:
                wt.append(float(i.split("kg")[0]))
            dataset.drop("Weight",axis=1,inplace=True)
            dataset["Weight(kg)"]=wt
        
        elif 'GB' in X[0]:
            for i in X:
                ram.append(int(i.split("GB")[0]))
            
            dataset.drop("Ram",axis=1,inplace=True)
            dataset["Ram(GB)"]=ram
    
        elif re.search(r"(\d{3,4}x\d{3,4})",X[0]):
            most_freq=dataset["ScreenResolution"].value_counts().keys()[0]
            for i in X:
                temp=(re.search(r"(\d{3,4}x\d{3,4})",i).group())
                temp2=i.split(temp)
                temp=temp.split('x')
                if temp2[0]=='':
                    scrn_type.append(most_freq)
                    x_res.append(int(temp[0]))
                    y_res.append(int(temp[1]))
                    all_res.append([None,int(temp[0]),int(temp[1])])
                else:
                    scrn_type.append(temp2[0])
                    x_res.append(int(temp[0]))
                    y_res.append(int(temp[1]))
                    all_res.append([temp2[0],int(temp[0]),int(temp[1])])
            dataset["X_Res"]=x_res
            dataset["Y_Res"]=y_res
            dataset["Screen_type"]=scrn_type
            dataset.drop("ScreenResolution",axis=1,inplace=True)
        elif 'GHz' in X[0]:
            for i in X:
                temp=re.search(r'(\d{1}.\d{0,2}GHz)',i)
                if temp:
                    temp0=re.split(r'(\d{1}.\d{0,2}GHz)',i)[0]
                    temp=temp.group().split('GHz')[0]
                    cpu_freq.append(temp)
                    cpu_type.append(temp[0])
                else:
                    temp0=re.split(r'(\d{1}GHz)',i)[1]
                    temp=re.search(r'(\d{1}GHz)',i).group().split('GHz')[0]
                    cpu_freq.append(temp)
                    cpu_type.append(temp0)
            dataset.drop("Cpu",axis=1,inplace=True)
            dataset["Cpu_freq(GHz)"]=cpu_freq
            dataset["Cpu_type(GHz)"]=cpu_type
    return dataset

In [30]:
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.pipeline import make_pipeline,FunctionTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.impute import KNNImputer

In [96]:
to_check=laptop_data.copy()

In [97]:
to_check=transformer(to_check)

TypeError: 'DataFrame' object is not callable